In [ ]:
!pip install numpy


In [ ]:
!pip install matplotlib


In [ ]:
!pip install ultralytics


In [ ]:
!pip install deep_sort_realtime

In [ ]:
!pip install opencv-python-headless


In [ ]:
!pip install python-firebase

In [ ]:
!pip install firebase-admin


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime
import csv
import os

# Firebase setup
app_name = 'sheep_monitoring_app'
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")  # Ensure the credentials file is in the same directory
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase
def update_sheep_data(sheep_id, coordinates, speed, health_status):
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)
    data = {
        "coordinates": coordinates,
        "speed": speed,
        "health_status": health_status,
        "timestamp": datetime.datetime.utcnow().isoformat()
    }
    sheep_ref.set(data)

# Initialize YOLO and DeepSORT
model_path = '/kaggle/input/detection-model-best/best.pt'  # Place the model file in the same directory
model = YOLO(model_path)
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

# Video settings
video_path = '/kaggle/input/sheep-video/Sheepeating.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_video_path = 'processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSV file setup
csv_path = 'tracking_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sheep ID', 'X Coordinate', 'Y Coordinate', 'Speed', 'Health Status', 'Timestamp'])

    # Tracking setup
    sheep_tracks = defaultdict(list)
    prev_positions = {}
    abnormal_ids = set()
    NORMAL_SPEED_MIN = 0
    NORMAL_SPEED_MAX = 1.5

    # Process video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Detect sheep
        results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
        detections = []
        for result in results:
            for box in result.boxes:
                try:
                    x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    if confidence > 0.5 and class_id == 0:  # Sheep class
                        detections.append([[x1, y1, x2, y2], confidence])
                except Exception as e:
                    print(f"Error processing box: {e}")
                    continue

        # Update tracker
        tracks = tracker.update_tracks(detections, frame=frame)
        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x, y, w, h = track.to_ltwh()
            center_x = int(x + w / 2)
            center_y = int(y + h / 2)

            # Calculate speed
            speed = 0
            if track_id in prev_positions:
                prev_x, prev_y = prev_positions[track_id]
                speed = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)

                # Check for abnormal speeds
                if speed < NORMAL_SPEED_MIN or speed > NORMAL_SPEED_MAX:
                    abnormal_ids.add(track_id)

            prev_positions[track_id] = (center_x, center_y)
            health_status = "abnormal" if track_id in abnormal_ids else "normal"

            # Write to CSV
            writer.writerow([track_id, center_x, center_y, speed, health_status, datetime.datetime.utcnow().isoformat()])

            # Update Firebase
            update_sheep_data(track_id, f"{center_x},{center_y}", speed, health_status)

            # Draw bounding box
            color = (0, 0, 255) if track_id in abnormal_ids else (0, 255, 0)
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(frame)

# Release resources
cap.release()
out.release()
print(f"Processed video saved at: {output_video_path}")
print(f"Tracking data saved at: {csv_path}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime
import csv
import os
from scipy.spatial import ConvexHull

# Firebase setup
app_name = 'sheep_monitoring_app'
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")  # Ensure the credentials file is in the same directory
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase
def update_sheep_data(sheep_id, coordinates, speed, health_status, avg_distance, area, stop_count, distance_from_group):
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)
    data = {
        "coordinates": coordinates,
        "speed": speed,
        "health_status": health_status,
        "avg_distance": avg_distance,
        "area": area,
        "stop_count": stop_count,
        "distance_from_group": distance_from_group,
        "timestamp": datetime.datetime.utcnow().isoformat()
    }
    sheep_ref.set(data)

# Initialize YOLO and DeepSORT
model_path = '/kaggle/input/detection-model-best/best.pt'  # Place the model file in the same directory
model = YOLO(model_path)
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

# Video settings
video_path = '/kaggle/input/sheep-video/Sheepeating.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_video_path = 'processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSV file setup
csv_path = 'tracking_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sheep ID', 'X Coordinate', 'Y Coordinate', 'Speed', 'Health Status', 'Timestamp'])

    # Tracking setup
    sheep_tracks = defaultdict(list)
    prev_positions = {}
    abnormal_ids = set()
    NORMAL_SPEED_MIN = 0  # Grazing sheep can stay stationary
    NORMAL_SPEED_MAX = 1.5  # Maximum grazing speed
    grazing_time = defaultdict(int)  # To track grazing-like behavior duration

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Detect sheep
        results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
        detections = []
        for result in results:
            for box in result.boxes:
                try:
                    x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    if confidence > 0.5 and class_id == 0:  # Sheep class
                        detections.append([[x1, y1, x2, y2], confidence])
                except Exception as e:
                    print(f"Error processing box: {e}")
                    continue

        # Update tracker
        tracks = tracker.update_tracks(detections, frame=frame)
        group_centroids = []  # For calculating group behavior
        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x, y, w, h = track.to_ltwh()
            center_x = int(x + w / 2)
            center_y = int(y + h / 2)

            # Track movement
            sheep_tracks[track_id].append((center_x, center_y))
            group_centroids.append((center_x, center_y))

            # Calculate speed
            speed = 0
            if track_id in prev_positions:
                prev_x, prev_y = prev_positions[track_id]
                speed = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)

            prev_positions[track_id] = (center_x, center_y)

            # Check movement patterns over time
            trajectory = sheep_tracks[track_id]
            distances = []
            if len(trajectory) > 10:  # Use the last 10 movements
                distances = [
                    np.sqrt((trajectory[i][0] - trajectory[i - 1][0]) ** 2 +
                            (trajectory[i][1] - trajectory[i - 1][1]) ** 2)
                    for i in range(1, len(trajectory))
                ]
                avg_distance = sum(distances) / len(distances)
            else:
                avg_distance = 0

            # Check area covered by movement
            if len(trajectory) > 3:
                points = np.array(trajectory)
                hull = ConvexHull(points)
                area = hull.volume  # Approximate area covered
            else:
                area = 0

            # Check frequency of stops
            stop_count = sum(1 for d in distances if d < 0.5) if len(trajectory) > 1 else 0

            # Group behavior: calculate deviation from group centroid
            if group_centroids:
                group_centroid_x = np.mean([pos[0] for pos in group_centroids])
                group_centroid_y = np.mean([pos[1] for pos in group_centroids])
                distance_from_group = np.sqrt((center_x - group_centroid_x) ** 2 + (center_y - group_centroid_y) ** 2)
            else:
                distance_from_group = 0

            # Combine multiple indicators
            grazing_score = 0
            if avg_distance < 2:  # Small average movement
                grazing_score += 1
            if area < 100:  # Small area indicates grazing
                grazing_score += 1
            if stop_count > 5:  # Frequent stops indicate grazing
                grazing_score += 1
            if distance_from_group < 50:  # Grazing sheep stay near the group
                grazing_score += 1

            # Determine health status
            health_status = "normal" if grazing_score >= 3 else "abnormal"

            # Write to CSV
            writer.writerow([track_id, center_x, center_y, speed, health_status, datetime.datetime.utcnow().isoformat()])

            # Update Firebase
            update_sheep_data(track_id, f"{center_x},{center_y}", speed, health_status, avg_distance, area, stop_count, distance_from_group)

            # Draw bounding box
            color = (0, 0, 255) if health_status == "abnormal" else (0, 255, 0)
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(frame)

# Release resources
cap.release()
out.release()
print(f"Processed video saved at: {output_video_path}")
print(f"Tracking data saved at: {csv_path}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, deque
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime
import csv
import os
from scipy.spatial import ConvexHull
from sklearn.cluster import DBSCAN

# Firebase setup
app_name = 'sheep_monitoring_app'
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)
app = firebase_admin.get_app(name=app_name)

def update_sheep_data(sheep_id, coordinates, speed, health_status, avg_distance, area, stop_count, distance_from_group):
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)
    data = {
        "coordinates": coordinates,
        "speed": speed,
        "health_status": health_status,
        "avg_distance": avg_distance,
        "area": area,
        "stop_count": stop_count,
        "distance_from_group": distance_from_group,
        "timestamp": datetime.datetime.utcnow().isoformat()
    }
    sheep_ref.set(data)

# Initialize YOLO and DeepSORT
model_path = '/kaggle/input/detection-model-best/best.pt'
model = YOLO(model_path)
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

# Video settings
video_path = '/kaggle/input/sheep-video/Sheepeating.mp4'
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_video_path = 'processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSV file setup
csv_path = 'tracking_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sheep ID', 'X Coordinate', 'Y Coordinate', 'Speed', 'Health Status', 'Timestamp'])

    sheep_tracks = defaultdict(list)
    prev_positions = {}
    speed_history = defaultdict(lambda: deque(maxlen=30))
    grazing_history = defaultdict(lambda: deque(maxlen=30))
    group_deviation_history = defaultdict(lambda: deque(maxlen=30))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Detect sheep
        results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
        detections = []
        for result in results:
            for box in result.boxes:
                try:
                    x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    if confidence > 0.5 and class_id == 0:
                        detections.append([[x1, y1, x2, y2], confidence])
                except Exception as e:
                    print(f"Error processing box: {e}")
                    continue

        # Update tracker
        tracks = tracker.update_tracks(detections, frame=frame)
        group_centroids = []  # For calculating group behavior
        all_positions = []

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x, y, w, h = track.to_ltwh()
            center_x = int(x + w / 2)
            center_y = int(y + h / 2)

            sheep_tracks[track_id].append((center_x, center_y))
            group_centroids.append((center_x, center_y))
            all_positions.append([center_x, center_y])

            # Calculate speed
            speed = 0
            if track_id in prev_positions:
                prev_x, prev_y = prev_positions[track_id]
                speed = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)

            prev_positions[track_id] = (center_x, center_y)

            # Check historical trends
            speed_history[track_id].append(speed)
            avg_speed = np.mean(speed_history[track_id])

            trajectory = sheep_tracks[track_id]
            distances = [
                np.sqrt((trajectory[i][0] - trajectory[i - 1][0]) ** 2 +
                        (trajectory[i][1] - trajectory[i - 1][1]) ** 2)
                for i in range(1, len(trajectory))
            ] if len(trajectory) > 1 else []

            avg_distance = sum(distances) / len(distances) if distances else 0
            area = ConvexHull(np.array(trajectory)).volume if len(trajectory) > 3 else 0
            stop_count = sum(1 for d in distances if d < 0.5)

            # Clustering using DBSCAN
            if len(all_positions) > 1:
                clustering = DBSCAN(eps=50, min_samples=2).fit(all_positions)
                cluster_label = clustering.labels_[group_centroids.index((center_x, center_y))]
                distance_from_group = 0 if cluster_label == -1 else np.linalg.norm(np.array([center_x, center_y]) - np.mean(np.array(all_positions), axis=0))
            else:
                distance_from_group = 0

            group_deviation_history[track_id].append(distance_from_group)
            avg_group_deviation = np.mean(group_deviation_history[track_id])

            grazing_score = 0
            if avg_distance < 2: grazing_score += 1
            if area < 100: grazing_score += 1
            if stop_count > 5: grazing_score += 1
            if distance_from_group < 50: grazing_score += 1
            grazing_history[track_id].append(grazing_score)

            # Determine health status
            health_status = "normal" if grazing_score >= 3 else "abnormal"

            # Write to CSV
            writer.writerow([track_id, center_x, center_y, speed, health_status, datetime.datetime.utcnow().isoformat()])

            # Update Firebase
            update_sheep_data(track_id, f"{center_x},{center_y}", speed, health_status, avg_distance, area, stop_count, distance_from_group)

            # Visualization
            color = (0, 0, 255) if health_status == "abnormal" else (0, 255, 0)
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            cv2.putText(frame, f"Health: {health_status}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            cv2.putText(frame, f"Grazing: {grazing_score}", (int(x), int(y) + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Add legend
        cv2.rectangle(frame, (10, 10), (300, 100), (255, 255, 255), -1)
        cv2.putText(frame, "Legend:", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(frame, "Green: Normal", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.putText(frame, "Red: Abnormal", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        out.write(frame)

cap.release()
out.release()
print(f"Processed video saved at: {output_video_path}")
print(f"Tracking data saved at: {csv_path}")

In [ ]:
#code for raspberry pi
import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import firebase_admin
from firebase_admin import credentials, db
import datetime
import threading
import queue
import time
import logging
import os

class SheepTracker:
    def __init__(self, 
                 model_path='/home/pi/models/best.pt',  # Adjusted for Raspberry Pi
                 database_url='https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com',
                 credentials_path='/home/pi/firebase-credentials.json'):
        """
        Initialize Raspberry Pi Sheep Tracking System
        
        Args:
            model_path (str): Path to custom YOLO model
            database_url (str): Firebase Realtime Database URL
            credentials_path (str): Path to Firebase credentials
        """
        # Logging setup
        logging.basicConfig(
            level=logging.INFO, 
            format='%(asctime)s - %(levelname)s: %(message)s',
            filename='/home/pi/sheep_tracking.log'
        )
        self.logger = logging.getLogger(__name__)

        # Firebase initialization (with error handling)
        try:
            if not firebase_admin._apps:
                cred = credentials.Certificate(credentials_path)
                firebase_admin.initialize_app(cred, {
                    'databaseURL': database_url
                })
        except Exception as e:
            self.logger.error(f"Firebase initialization error: {e}")
            raise

        # Model and tracking initialization
        try:
            self.model = YOLO(model_path)
            self.tracker = DeepSort(
                max_age=30, 
                n_init=3, 
                max_iou_distance=0.7,
                nn_budget=50  # Reduced for Raspberry Pi
            )
        except Exception as e:
            self.logger.error(f"Model initialization error: {e}")
            raise

        # Processing queues for multi-threading
        self.frame_queue = queue.Queue(maxsize=10)
        self.results_queue = queue.Queue(maxsize=10)

        # Tracking variables
        self.sheep_tracks = {}
        self.prev_positions = {}

    def _detect_and_track(self):
        """
        Detect and track sheep in frames from the queue
        """
        while True:
            try:
                frame = self.frame_queue.get(timeout=5)
                
                # Detect sheep
                results = self.model.predict(
                    frame, 
                    stream=True, 
                    conf=0.5, 
                    iou=0.45, 
                    device='cpu'  # Ensure CPU processing
                )
                
                detections = []
                for result in results:
                    for box in result.boxes:
                        try:
                            x1, y1, x2, y2 = map(int, box.xyxy[0])
                            confidence = float(box.conf[0])
                            class_id = int(box.cls[0])
                            
                            if confidence > 0.5 and class_id == 0:
                                detections.append([[x1, y1, x2, y2], confidence])
                        except Exception as e:
                            self.logger.warning(f"Detection processing error: {e}")

                # Track detected sheep
                tracks = self.tracker.update_tracks(detections, frame=frame)
                
                # Process tracking results
                processed_tracks = self._process_tracks(tracks, frame)
                
                # Put results in queue
                if not self.results_queue.full():
                    self.results_queue.put(processed_tracks)
                
                self.frame_queue.task_done()
            
            except queue.Empty:
                time.sleep(0.1)
            except Exception as e:
                self.logger.error(f"Tracking error: {e}")

    def _process_tracks(self, tracks, frame):
        """
        Process tracking results and compute sheep metrics
        
        Args:
            tracks (list): Tracking results
            frame (np.ndarray): Current video frame
        
        Returns:
            dict: Processed sheep tracking information
        """
        processed_tracks = {}
        
        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x, y, w, h = track.to_ltwh()
            center_x, center_y = int(x + w/2), int(y + h/2)

            # Speed calculation
            speed = 0
            if track_id in self.prev_positions:
                prev_x, prev_y = self.prev_positions[track_id]
                speed = np.sqrt((center_x - prev_x)**2 + (center_y - prev_y)**2)
            
            self.prev_positions[track_id] = (center_x, center_y)

            # Health and behavior assessment
            health_status = self._assess_health(speed, center_x, center_y)

            # Store track information
            processed_tracks[track_id] = {
                'coordinates': f"{center_x},{center_y}",
                'speed': speed,
                'health_status': health_status,
                'timestamp': datetime.datetime.utcnow().isoformat()
            }

        return processed_tracks

    def _assess_health(self, speed, x, y):
        """
        Basic health assessment based on movement
        
        Args:
            speed (float): Movement speed
            x (int): X coordinate
            y (int): Y coordinate
        
        Returns:
            str: Health status
        """
        # Simple health assessment logic
        if speed < 1 and 200 < x < 600 and 300 < y < 700:
            return "potential_issue"
        return "normal"

    def _upload_to_firebase(self, processed_tracks):
        """
        Upload processed tracking data to Firebase
        
        Args:
            processed_tracks (dict): Processed tracking information
        """
        try:
            for sheep_id, data in processed_tracks.items():
                sheep_ref = db.reference(f'sheep/{sheep_id}')
                sheep_ref.set(data)
        except Exception as e:
            self.logger.error(f"Firebase upload error: {e}")

    def start_tracking(self):
        """
        Start real-time tracking process
        """
        # Camera setup (USB camera)
        try:
            cap = cv2.VideoCapture(0)  # Use USB camera
            cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
            cap.set(cv2.CAP_PROP_FPS, 30)
        except Exception as e:
            self.logger.error(f"Camera initialization error: {e}")
            return

        # Multi-threaded processing
        detection_thread = threading.Thread(target=self._detect_and_track, daemon=True)
        upload_thread = threading.Thread(target=self._upload_monitoring_data, daemon=True)
        
        detection_thread.start()
        upload_thread.start()

        while True:
            ret, frame = cap.read()
            if not ret:
                self.logger.warning("Failed to capture frame")
                time.sleep(0.1)
                continue

            # Put frame in queue for processing
            if not self.frame_queue.full():
                self.frame_queue.put(frame)

            # Optional: Display frame for local monitoring
            cv2.imshow('Sheep Tracking', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Cleanup
        cap.release()
        cv2.destroyAllWindows()

    def _upload_monitoring_data(self):
        """
        Continuously upload processed tracking data
        """
        while True:
            try:
                # Get latest processed tracks
                processed_tracks = self.results_queue.get(timeout=5)
                
                # Upload to Firebase
                self._upload_to_firebase(processed_tracks)
                
                self.results_queue.task_done()
            
            except queue.Empty:
                time.sleep(0.1)
            except Exception as e:
                self.logger.error(f"Monitoring data upload error: {e}")

def main():
    """
    Main execution function
    """
    try:
        # Ensure model and credentials paths are correct
        tracker = SheepTracker()
        tracker.start_tracking()
    
    except Exception as e:
        logging.error(f"Tracking process failed: {e}")

if __name__ == "__main__":
    main()

In [ ]:
!pip install torch


In [ ]:
import torch

# Save the DeepSORT tracker state
tracker_state_path = 'deep_sort_tracker_state.pth'
torch.save(tracker.state_dict(), tracker_state_path)


In [ ]:
# Save the DeepSORT tracker state
tracker_state_path = 'deep_sort_tracker_state.pth'
torch.save(tracker.state_dict(), tracker_state_path)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime
import csv
import os
from scipy.spatial import ConvexHull

# Conversion factor
PIXELS_PER_CM = 37.795275591

# Firebase setup (remains the same)
app_name = 'sheep_monitoring_app'
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase (modified to use cm)
def update_sheep_data(sheep_id, coordinates, health_status, avg_distance, area, stop_count, distance_from_group, notifications):
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)
    data = {
        "coordinates": coordinates,
        "notifications": notifications,
        "health_status": health_status,
        "avg_distance_cm": avg_distance,  # Now in cm
        "area_cm2": area,  # Now in cm²
        "stop_count": stop_count,
        "distance_from_group_cm": distance_from_group,  # Now in cm
        "timestamp": datetime.datetime.utcnow().isoformat()
    }
    sheep_ref.set(data)

# Rest of the initialization remains the same (YOLO, DeepSORT, video capture)
model_path = '/kaggle/input/detection-model-best/best.pt'
model = YOLO(model_path)
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

video_path = '/kaggle/input/sheepv/Sheep.mp4'
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_video_path = 'processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSV file setup
csv_path = 'tracking_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sheep ID', 'X Coordinate', 'Y Coordinate', 'Health Status', 'Timestamp'])

    # Tracking setup
    sheep_tracks = defaultdict(list)
    sheep_distances = defaultdict(list)
    prev_positions = {}
    abnormal_ids = set()
    grazing_time = defaultdict(int)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Detection and tracking code remains mostly the same
        results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
        detections = []
        for result in results:
            for box in result.boxes:
                try:
                    x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    if confidence > 0.5 and class_id == 0:  # Sheep class
                        detections.append([[x1, y1, x2, y2], confidence])
                except Exception as e:
                    print(f"Error processing box: {e}")
                    continue

        tracks = tracker.update_tracks(detections, frame=frame)
        group_centroids = []
        notifications = []

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x, y, w, h = track.to_ltwh()
            center_x = int(x + w / 2)
            center_y = int(y + h / 2)

            sheep_tracks[track_id].append((center_x, center_y))
            group_centroids.append((center_x, center_y))

            # Stationary time detection
            if track_id in prev_positions:
                prev_x, prev_y, stationary_time = prev_positions[track_id]
                if center_x == prev_x and center_y == prev_y:
                    stationary_time += 1
                    if stationary_time >= fps * 10:
                        notifications.append(f"Sheep ID {track_id} has been stationary for more than 10 seconds.")
                        health_status = "abnormal"
                else:
                    stationary_time = 0
                prev_positions[track_id] = (center_x, center_y, stationary_time)
            else:
                prev_positions[track_id] = (center_x, center_y, 0)

            # Calculate metrics in cm
            trajectory = sheep_tracks[track_id]
            if len(trajectory) > 10:
                # Distances in cm
                distances = [
                    np.sqrt((trajectory[i][0] - trajectory[i-1][0])**2 + 
                            (trajectory[i][1] - trajectory[i-1][1])**2) / PIXELS_PER_CM
                    for i in range(1, len(trajectory))
                ]
                sheep_distances[track_id] = distances
                avg_distance = sum(distances) / len(distances)

                # Area in cm²
                points = np.array(trajectory)
                hull = ConvexHull(points)
                area = hull.area / (PIXELS_PER_CM ** 2)

                # Stop count with cm threshold
                stop_count = sum(1 for d in distances if d < 0.5)

                # Distance from group in cm
                if group_centroids:
                    group_centroid_x = np.mean([pos[0] for pos in group_centroids])
                    group_centroid_y = np.mean([pos[1] for pos in group_centroids])
                    distance_from_group = np.sqrt(
                        (center_x - group_centroid_x)**2 + 
                        (center_y - group_centroid_y)**2
                    ) / PIXELS_PER_CM
                else:
                    distance_from_group = 0

                # Health status logic
                grazing_score = 0
                if avg_distance < 2:  # cm
                    grazing_score += 1
                if area < 100:  # cm²
                    grazing_score += 1
                if stop_count > 5:
                    grazing_score += 1
                if distance_from_group < 50:  # cm
                    grazing_score += 1

                health_status = "normal" if grazing_score >= 3 else "abnormal"

                # Write to CSV with cm measurements
                writer.writerow([
                    track_id, 
                    center_x / PIXELS_PER_CM,  # x in cm 
                    center_y / PIXELS_PER_CM,  # y in cm
                    health_status, 
                    datetime.datetime.utcnow().isoformat()
                ])

                # Update Firebase with cm measurements
                update_sheep_data(
                    track_id, 
                    f"{center_x/PIXELS_PER_CM},{center_y/PIXELS_PER_CM}", 
                    health_status, 
                    avg_distance, 
                    area, 
                    stop_count, 
                    distance_from_group, 
                    notifications
                )

            # Drawing remains the same
            color = (0, 0, 255) if health_status == "abnormal" else (0, 255, 0)
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(frame)

# Release resources
cap.release()
out.release()

# Visualization with cm units
plt.figure(figsize=(15, 10))

# Calculate metrics for visualization
track_metrics = {}
for track_id, coordinates in sheep_tracks.items():
    if len(coordinates) > 10:
        # CM calculations (same as in tracking)
        distances = [
            np.sqrt((coordinates[i][0] - coordinates[i-1][0])**2 + 
                    (coordinates[i][1] - coordinates[i-1][1])**2) / PIXELS_PER_CM
            for i in range(1, len(coordinates))
        ]
        points = np.array(coordinates)
        hull = ConvexHull(points)
        
        track_metrics[track_id] = {
            'avg_distance': np.mean(distances),
            'area': hull.area / (PIXELS_PER_CM ** 2),
            'stop_count': sum(1 for dist in distances if dist < 0.5)
        }

# Plotting with updated units
plt.subplot(2, 2, 1)
plt.bar(range(len(track_metrics)), [metric['avg_distance'] for metric in track_metrics.values()], color='blue')
plt.title("Average Movement Distance")
plt.xlabel("Sheep ID")
plt.ylabel("Avg Distance (cm)")

plt.subplot(2, 2, 2)
plt.bar(range(len(track_metrics)), [metric['area'] for metric in track_metrics.values()], color='green')
plt.title("Area Covered")
plt.xlabel("Sheep ID")
plt.ylabel("Total Area Covered (cm²)")

plt.subplot(2, 2, 3)
plt.bar(range(len(track_metrics)), [metric['stop_count'] for metric in track_metrics.values()], color='orange')
plt.title("Frequency of Stops")
plt.xlabel("Sheep ID")
plt.ylabel("Stop Count")

plt.tight_layout()
plt.savefig('sheep_tracking_metrics.png')
plt.close()

print(f"Processed video saved at: {output_video_path}")
print(f"Tracking data saved at: {csv_path}")
print("Metrics visualization saved as 'sheep_tracking_metrics.png'")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime
import csv
import os
from scipy.spatial import ConvexHull

# Conversion factor
PIXELS_PER_CM = 37.795275591

# Firebase setup 
app_name = 'sheep_monitoring_app'
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase (modified to use cm)
def update_sheep_data(sheep_id, coordinates, health_status, avg_distance, area, stop_count, distance_from_group, notifications):
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)
    data = {
        "coordinates": coordinates,
        "notifications": notifications,
        "health_status": health_status,
        "avg_distance_cm": avg_distance,  
        "area_cm2": area,  
        "stop_count": stop_count,
        "distance_from_group_cm": distance_from_group,  
        "timestamp": datetime.datetime.utcnow().isoformat()
    }
    sheep_ref.update(data)

# Rest of the initialization remains the same (YOLO, DeepSORT, video capture)
model_path = '/kaggle/input/detection-model-best/best.pt'
model = YOLO(model_path)
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

video_path = '/kaggle/input/manuel/maNUEL.mp4'
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_video_path = 'processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSV file setup
csv_path = 'tracking_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sheep ID', 'X Coordinate', 'Y Coordinate', 'Health Status', 'Timestamp'])

    # Tracking setup
    sheep_tracks = defaultdict(list)
    sheep_distances = defaultdict(list)
    prev_positions = {}
    abnormal_ids = set()
    grazing_time = defaultdict(int)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Detection and tracking code remains mostly the same
        results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
        detections = []
        for result in results:
            for box in result.boxes:
                try:
                    x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    if confidence > 0.5 and class_id == 0:  # Sheep class
                        detections.append([[x1, y1, x2, y2], confidence])
                except Exception as e:
                    print(f"Error processing box: {e}")
                    continue

        tracks = tracker.update_tracks(detections, frame=frame)
        group_centroids = []
        notifications = []
        health_status = "normal"  # Initialize health_status as normal

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x, y, w, h = track.to_ltwh()
            center_x = int(x + w / 2)
            center_y = int(y + h / 2)

            sheep_tracks[track_id].append((center_x, center_y))
            group_centroids.append((center_x, center_y))

            # Stationary time detection
            if track_id in prev_positions:
                prev_x, prev_y, stationary_time = prev_positions[track_id]
                if center_x == prev_x and center_y == prev_y:
                    stationary_time += 1
                    if stationary_time >= fps * 30:
                        notifications.append(f"Sheep ID {track_id} has been stationary for more than 30 seconds.")
                        print("Notifications:", notifications)
                        health_status = "abnormal"
                else:
                    stationary_time = 0
                prev_positions[track_id] = (center_x, center_y, stationary_time)
            else:
                prev_positions[track_id] = (center_x, center_y, 0)

            # Calculate metrics in cm
            trajectory = sheep_tracks[track_id]
            if len(trajectory) > 10:
                # Distances in cm
                distances = [
                    np.sqrt((trajectory[i][0] - trajectory[i-1][0])**2 + 
                            (trajectory[i][1] - trajectory[i-1][1])**2) / PIXELS_PER_CM
                    for i in range(1, len(trajectory))
                ]
                sheep_distances[track_id] = distances
                avg_distance = sum(distances) / len(distances)

                # Area in cm²
                points = np.array(trajectory)
                hull = ConvexHull(points)
                area = hull.area / (PIXELS_PER_CM ** 2)

                # Stop count with cm threshold
                stop_count = sum(1 for d in distances if d < 0.5)

                # Distance from group in cm
                if group_centroids:
                    group_centroid_x = np.mean([pos[0] for pos in group_centroids])
                    group_centroid_y = np.mean([pos[1] for pos in group_centroids])
                    distance_from_group = np.sqrt(
                        (center_x - group_centroid_x)**2 + 
                        (center_y - group_centroid_y)**2
                    ) / PIXELS_PER_CM
                else:
                    distance_from_group = 0

                # Health status logic
                grazing_score = 0
                if avg_distance < 2:  # cm
                    grazing_score += 1
                if area < 100:  # cm²
                    grazing_score += 1
                if stop_count > 5:
                    grazing_score += 1
                if distance_from_group < 50:  # cm
                    grazing_score += 1

                health_status = "normal" if grazing_score >= 3 else "abnormal"

                # Write to CSV with cm measurements
                writer.writerow([
                    track_id, 
                    center_x / PIXELS_PER_CM,  # x in cm 
                    center_y / PIXELS_PER_CM,  # y in cm
                    health_status, 
                    datetime.datetime.utcnow().isoformat()
                ])

                # Update Firebase with cm measurements
                update_sheep_data(
                    track_id, 
                    f"{center_x/PIXELS_PER_CM},{center_y/PIXELS_PER_CM}", 
                    health_status, 
                    avg_distance, 
                    area, 
                    stop_count, 
                    distance_from_group, 
                    notifications
                )

            # Drawing remains the same
            color = (0, 0, 255) if health_status == "abnormal" else (0, 255, 0)
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(frame)

# Release resources
cap.release()
out.release()

# Visualization with cm units
plt.figure(figsize=(15, 10))

# Calculate metrics for visualization
track_metrics = {}
for track_id, coordinates in sheep_tracks.items():
    if len(coordinates) > 10:
        # CM calculations (same as in tracking)
        distances = [
            np.sqrt((coordinates[i][0] - coordinates[i-1][0])**2 + 
                    (coordinates[i][1] - coordinates[i-1][1])**2) / PIXELS_PER_CM
            for i in range(1, len(coordinates))
        ]
        points = np.array(coordinates)
        hull = ConvexHull(points)
        
        track_metrics[track_id] = {
            'avg_distance': np.mean(distances),
            'area': hull.area / (PIXELS_PER_CM ** 2),
            'stop_count': sum(1 for dist in distances if dist < 0.5)
        }

# Plotting with updated units
plt.subplot(2, 2, 1)
plt.bar(range(len(track_metrics)), [metric['avg_distance'] for metric in track_metrics.values()], color='blue')
plt.title("Average Movement Distance")
plt.xlabel("Sheep ID")
plt.ylabel("Avg Distance (cm)")

plt.subplot(2, 2, 2)
plt.bar(range(len(track_metrics)), [metric['area'] for metric in track_metrics.values()], color='green')
plt.title("Area Covered")
plt.xlabel("Sheep ID")
plt.ylabel("Total Area Covered (cm²)")

plt.subplot(2, 2, 3)
plt.bar(range(len(track_metrics)), [metric['stop_count'] for metric in track_metrics.values()], color='orange')
plt.title("Frequency of Stops")
plt.xlabel("Sheep ID")
plt.ylabel("Stop Count")

plt.tight_layout()
plt.savefig('sheep_tracking_metrics.png')
plt.close()

print(f"Processed video saved at: {output_video_path}")
print(f"Tracking data saved at: {csv_path}")
print("Metrics visualization saved as 'sheep_tracking_metrics.png'")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime
import csv
from scipy.spatial import ConvexHull

# Conversion factor
PIXELS_PER_CM = 37.795275591

# Firebase setup
app_name = 'sheep_monitoring_app'
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase
def update_sheep_data(sheep_id, coordinates, health_status, avg_distance, area, stop_count, distance_from_group, notifications, activity_level):
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)
    data = {
        "coordinates": coordinates,
        "notifications": notifications,
        "health_status": health_status,
        "avg_distance_cm": avg_distance,
        "area_cm2": area,
        "stop_count": stop_count,
        "distance_from_group_cm": distance_from_group,
        "timestamp": datetime.datetime.utcnow().isoformat(),
        "activity_level": activity_level  # Save the activity level
    }
    sheep_ref.update(data)

# Function to classify sheep activity
def classify_activity(avg_distance_cm, area_cm2, distance_from_group_cm):
    if avg_distance_cm < 0.1 and area_cm2 < 0.1:
        activity = "inactive"
    elif 0.1 <= avg_distance_cm < 0.8 and 0.1 <= area_cm2 < 0.8:
        activity = "active"
    else:
        activity = "very active"
    
    # Refine classification for isolation
    if distance_from_group_cm > 10:
        activity = "inactive"
    
    return activity

# Model setup
model_path = '/kaggle/input/detection-model-best/best.pt'
model = YOLO(model_path)
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

# Video input and output setup
video_path = '/kaggle/input/gazing/WhatsApp Video 2024-12-18 at 09.32.55.mp4'
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_video_path = 'processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSV file setup
csv_path = 'tracking_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sheep ID', 'X Coordinate', 'Y Coordinate', 'Health Status', 'Activity Level', 'Timestamp'])

# Tracking setup
sheep_tracks = defaultdict(list)
sheep_distances = defaultdict(list)
prev_positions = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
    detections = []
    for result in results:
        for box in result.boxes:
            try:
                x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                confidence = float(box.conf[0])
                class_id = int(box.cls[0])
                if confidence > 0.5 and class_id == 0:  # Sheep class
                    detections.append([[x1, y1, x2, y2], confidence])
            except Exception as e:
                print(f"Error processing box: {e}")
                continue

    tracks = tracker.update_tracks(detections, frame=frame)
    group_centroids = []

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        x, y, w, h = track.to_ltwh()
        center_x = int(x + w / 2)
        center_y = int(y + h / 2)

        sheep_tracks[track_id].append((center_x, center_y))
        group_centroids.append((center_x, center_y))

        # Calculate metrics in cm
        trajectory = sheep_tracks[track_id]
        if len(trajectory) > 10:
            distances = [
                np.sqrt((trajectory[i][0] - trajectory[i-1][0])**2 + 
                        (trajectory[i][1] - trajectory[i-1][1])**2) / PIXELS_PER_CM
                for i in range(1, len(trajectory))
            ]
            avg_distance = sum(distances) / len(distances)

            points = np.array(trajectory)
            hull = ConvexHull(points)
            area = hull.area / (PIXELS_PER_CM ** 2)

            if group_centroids:
                group_centroid_x = np.mean([pos[0] for pos in group_centroids])
                group_centroid_y = np.mean([pos[1] for pos in group_centroids])
                distance_from_group = np.sqrt(
                    (center_x - group_centroid_x)**2 + 
                    (center_y - group_centroid_y)**2
                ) / PIXELS_PER_CM
            else:
                distance_from_group = 0

            # Classify activity
            activity_level = classify_activity(avg_distance, area, distance_from_group)

            # Write to CSV
            writer.writerow([
                track_id,
                center_x / PIXELS_PER_CM,
                center_y / PIXELS_PER_CM,
                "normal",  # Replace with actual health status logic
                activity_level,
                datetime.datetime.utcnow().isoformat()
            ])

            # Update Firebase with activity level
            update_sheep_data(
                track_id,
                f"{center_x / PIXELS_PER_CM},{center_y / PIXELS_PER_CM}",
                "normal",  # Replace with actual health status logic
                avg_distance,
                area,
                stop_count=0,  # Replace if stop count is implemented
                distance_from_group=distance_from_group,
                notifications=[f"Activity: {activity_level}"],
                activity_level=activity_level  # Save activity level to Firebase
            )

            # Annotate frame
            cv2.putText(frame, f"Activity: {activity_level}", (int(x), int(y) + h + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Draw bounding box
        cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()

print(f"Processed video saved at: {output_video_path}")
print(f"Tracking data saved at: {csv_path}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime
import csv
import os
from scipy.spatial import ConvexHull

# Conversion factor
PIXELS_PER_CM = 37.795275591

# Firebase setup 
app_name = 'sheep_monitoring_app'
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase (modified to use cm)
def update_sheep_data(sheep_id, coordinates, health_status, avg_distance, area, stop_count, distance_from_group, notifications, activity_level):
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)
    data = {
        "coordinates": coordinates,
        "notifications": notifications,
        "health_status": health_status,
        "avg_distance_cm": avg_distance,  
        "area_cm2": area,  
        "stop_count": stop_count,
        "distance_from_group_cm": distance_from_group,  
        "activity_level": activity_level,  # Save activity level
        "timestamp": datetime.datetime.utcnow().isoformat()
    }
    sheep_ref.update(data)

# Function to classify activity level based on the thresholds
def classify_activity(avg_distance, area, distance_from_group):
    if avg_distance < 0.1 and area < 0.1 and distance_from_group > 10:
        return "Inactive"
    elif avg_distance >= 0.8 or area >= 0.8:
        return "Very Active"
    elif avg_distance >= 0.1 and avg_distance < 0.8 or area >= 0.1 and area < 0.8:
        return "Active"
    return "Inactive"

# Rest of the initialization remains the same (YOLO, DeepSORT, video capture)
model_path = '/kaggle/input/detection-model-best/best.pt'
model = YOLO(model_path)
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

video_path = '/kaggle/input/manuel/maNUEL.mp4'
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_video_path = 'processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSV file setup
csv_path = 'tracking_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sheep ID', 'X Coordinate', 'Y Coordinate', 'Health Status', 'Activity Level', 'Timestamp'])

    # Tracking setup
    sheep_tracks = defaultdict(list)
    sheep_distances = defaultdict(list)
    prev_positions = {}
    abnormal_ids = set()
    grazing_time = defaultdict(int)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Detection and tracking code remains mostly the same
        results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
        detections = []
        for result in results:
            for box in result.boxes:
                try:
                    x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    if confidence > 0.5 and class_id == 0:  # Sheep class
                        detections.append([[x1, y1, x2, y2], confidence])
                except Exception as e:
                    print(f"Error processing box: {e}")
                    continue

        tracks = tracker.update_tracks(detections, frame=frame)
        group_centroids = []
        notifications = []
        health_status = "normal"  # Initialize health_status as normal

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x, y, w, h = track.to_ltwh()
            center_x = int(x + w / 2)
            center_y = int(y + h / 2)

            sheep_tracks[track_id].append((center_x, center_y))
            group_centroids.append((center_x, center_y))

            # Stationary time detection
            if track_id in prev_positions:
                prev_x, prev_y, stationary_time = prev_positions[track_id]
                if center_x == prev_x and center_y == prev_y:
                    stationary_time += 1
                    if stationary_time >= fps * 30:
                        notifications.append(f"Sheep ID {track_id} has been stationary for more than 30 seconds.")
                        print("Notifications:", notifications)
                        health_status = "abnormal"
                else:
                    stationary_time = 0
                prev_positions[track_id] = (center_x, center_y, stationary_time)
            else:
                prev_positions[track_id] = (center_x, center_y, 0)

            # Calculate metrics in cm
            trajectory = sheep_tracks[track_id]
            if len(trajectory) > 10:
                # Distances in cm
                distances = [
                    np.sqrt((trajectory[i][0] - trajectory[i-1][0])**2 + 
                            (trajectory[i][1] - trajectory[i-1][1])**2) / PIXELS_PER_CM
                    for i in range(1, len(trajectory))
                ]
                sheep_distances[track_id] = distances
                avg_distance = sum(distances) / len(distances)

                # Area in cm²
                points = np.array(trajectory)
                hull = ConvexHull(points)
                area = hull.area / (PIXELS_PER_CM ** 2)

                # Stop count with cm threshold
                stop_count = sum(1 for d in distances if d < 0.5)

                # Distance from group in cm
                if group_centroids:
                    group_centroid_x = np.mean([pos[0] for pos in group_centroids])
                    group_centroid_y = np.mean([pos[1] for pos in group_centroids])
                    distance_from_group = np.sqrt(
                        (center_x - group_centroid_x)**2 + 
                        (center_y - group_centroid_y)**2
                    ) / PIXELS_PER_CM
                else:
                    distance_from_group = 0

                # Classify activity level
                activity_level = classify_activity(avg_distance, area, distance_from_group)

                # Write to CSV
                writer.writerow([
                    track_id,
                    center_x / PIXELS_PER_CM,
                    center_y / PIXELS_PER_CM,
                    health_status,
                    activity_level,
                    datetime.datetime.utcnow().isoformat()
                ])

                # Update Firebase with activity level
                update_sheep_data(
                    track_id,
                    f"{center_x / PIXELS_PER_CM},{center_y / PIXELS_PER_CM}",
                    health_status,
                    avg_distance,
                    area,
                    stop_count,
                    distance_from_group,
                    notifications,
                    activity_level
                )

            # Drawing remains the same
            color = (0, 0, 255) if health_status == "abnormal" else (0, 255, 0)
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(frame)

# Release resources
cap.release()
out.release()

print(f"Processed video saved at: {output_video_path}")
print(f"Tracking data saved at: {csv_path}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime
import csv
import os
from scipy.spatial import ConvexHull

# Conversion factor
PIXELS_PER_CM = 37.795275591

# Firebase setup 
app_name = 'sheep_monitoring_app'
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase (modified to use cm)
def update_sheep_data(sheep_id, coordinates, health_status, avg_distance, area, stop_count, distance_from_group, notifications, activity_level):
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)
    data = {
        "coordinates": coordinates,
        "notifications": notifications,
        "health_status": health_status,
        "avg_distance_cm": avg_distance,  
        "area_cm2": area,  
        "stop_count": stop_count,
        "distance_from_group_cm": distance_from_group,  
        "activity_level": activity_level,  # Save activity level
        "timestamp": datetime.datetime.utcnow().isoformat()
    }
    sheep_ref.update(data)

# Function to classify activity level based on the thresholds
def classify_activity(avg_distance, area, distance_from_group):
    if avg_distance < 0.1 and area < 0.1 and distance_from_group > 10:
        return "Inactive"
    elif avg_distance >= 0.8 or area >= 0.8:
        return "Very Active"
    elif avg_distance >= 0.1 and avg_distance < 0.8 or area >= 0.1 and area < 0.8:
        return "Active"
    return "Inactive"

# Rest of the initialization remains the same (YOLO, DeepSORT, video capture)
model_path = '/kaggle/input/modelbestpt/codesmodel.pt'
model = YOLO(model_path)
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

video_path = '/kaggle/input/gazing/WhatsApp Video 2024-12-18 at 09.32.55.mp4'
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_video_path = 'processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSV file setup
csv_path = 'tracking_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sheep ID', 'X Coordinate', 'Y Coordinate', 'Health Status', 'Activity Level', 'Timestamp'])

    # Tracking setup
    sheep_tracks = defaultdict(list)
    sheep_distances = defaultdict(list)
    prev_positions = {}
    abnormal_ids = set()
    grazing_time = defaultdict(int)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Detection and tracking code remains mostly the same
        results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
        detections = []
        for result in results:
            for box in result.boxes:
                try:
                    x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    if confidence > 0.5 and class_id == 0:  # Sheep class
                        detections.append([[x1, y1, x2, y2], confidence])
                except Exception as e:
                    print(f"Error processing box: {e}")
                    continue

        tracks = tracker.update_tracks(detections, frame=frame)
        group_centroids = []
        notifications = []
        health_status = "normal"  # Initialize health_status as normal

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x, y, w, h = track.to_ltwh()
            center_x = int(x + w / 2)
            center_y = int(y + h / 2)

            sheep_tracks[track_id].append((center_x, center_y))
            group_centroids.append((center_x, center_y))

            # Stationary time detection
            if track_id in prev_positions:
                prev_x, prev_y, stationary_time = prev_positions[track_id]
                if center_x == prev_x and center_y == prev_y:
                    stationary_time += 1
                    if stationary_time >= fps * 30:
                        notifications.append(f"Sheep ID {track_id} has been stationary for more than 30 seconds.")
                        print("Notifications:", notifications)
                        health_status = "abnormal"
                else:
                    stationary_time = 0
                prev_positions[track_id] = (center_x, center_y, stationary_time)
            else:
                prev_positions[track_id] = (center_x, center_y, 0)

            # Calculate metrics in cm
            trajectory = sheep_tracks[track_id]
            if len(trajectory) > 10:
                # Distances in cm
                distances = [
                    np.sqrt((trajectory[i][0] - trajectory[i-1][0])**2 + 
                            (trajectory[i][1] - trajectory[i-1][1])**2) / PIXELS_PER_CM
                    for i in range(1, len(trajectory))
                ]
                sheep_distances[track_id] = distances
                avg_distance = sum(distances) / len(distances)

                # Area in cm²
                points = np.array(trajectory)
                hull = ConvexHull(points)
                area = hull.area / (PIXELS_PER_CM ** 2)

                # Stop count with cm threshold
                stop_count = sum(1 for d in distances if d < 0.5)

                # Distance from group in cm
                if group_centroids:
                    group_centroid_x = np.mean([pos[0] for pos in group_centroids])
                    group_centroid_y = np.mean([pos[1] for pos in group_centroids])
                    distance_from_group = np.sqrt(
                        (center_x - group_centroid_x)**2 + 
                        (center_y - group_centroid_y)**2
                    ) / PIXELS_PER_CM
                else:
                    distance_from_group = 0

                # Classify activity level
                activity_level = classify_activity(avg_distance, area, distance_from_group)

                # Health status logic
                grazing_score = 0
                if avg_distance < 2:  # cm
                    grazing_score += 1
                if area < 100:  # cm²
                    grazing_score += 1
                if stop_count > 5:
                    grazing_score += 1
                if distance_from_group < 50:  # cm
                    grazing_score += 1

                health_status = "normal" if grazing_score >= 3 else "abnormal"

                # Write to CSV with cm measurements and activity level
                writer.writerow([
                    track_id, 
                    center_x / PIXELS_PER_CM,  # x in cm 
                    center_y / PIXELS_PER_CM,  # y in cm
                    health_status, 
                    activity_level,
                    datetime.datetime.utcnow().isoformat()
                ])

                # Update Firebase with cm measurements and activity level
                update_sheep_data(
                    track_id, 
                    f"{center_x/PIXELS_PER_CM},{center_y/PIXELS_PER_CM}", 
                    health_status, 
                    avg_distance, 
                    area, 
                    stop_count, 
                    distance_from_group, 
                    notifications,
                    activity_level
                )

            # Drawing remains the same
            color = (0, 0, 255) if health_status == "abnormal" else (0, 255, 0)
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(frame)

# Release resources
cap.release()
out.release()

# Visualization with cm units
plt.figure(figsize=(15, 10))

# Calculate metrics for visualization
track_metrics = {}
for track_id, coordinates in sheep_tracks.items():
    if len(coordinates) > 10:
        # CM calculations (same as in tracking)
        distances = [
            np.sqrt((coordinates[i][0] - coordinates[i-1][0])**2 + 
                    (coordinates[i][1] - coordinates[i-1][1])**2) / PIXELS_PER_CM
            for i in range(1, len(coordinates))
        ]
        points = np.array(coordinates)
        hull = ConvexHull(points)
        
        track_metrics[track_id] = {
            'avg_distance': np.mean(distances),
            'area': hull.area / (PIXELS_PER_CM ** 2),
            'stop_count': sum(1 for dist in distances if dist < 0.5)
        }

# Plotting with updated units
plt.subplot(2, 2, 1)
plt.bar(range(len(track_metrics)), [metric['avg_distance'] for metric in track_metrics.values()], color='blue')
plt.title("Average Movement Distance")
plt.xlabel("Sheep ID")
plt.ylabel("Avg Distance (cm)")

plt.subplot(2, 2, 2)
plt.bar(range(len(track_metrics)), [metric['area'] for metric in track_metrics.values()], color='green')
plt.title("Area Covered")
plt.xlabel("Sheep ID")
plt.ylabel("Total Area Covered (cm²)")

plt.subplot(2, 2, 3)
plt.bar(range(len(track_metrics)), [metric['stop_count'] for metric in track_metrics.values()], color='orange')
plt.title("Frequency of Stops")
plt.xlabel("Sheep ID")
plt.ylabel("Stop Count")

plt.tight_layout()
plt.savefig('sheep_tracking_metrics.png')
plt.close()

print(f"Processed video saved at: {output_video_path}")
print(f"Tracking data saved at: {csv_path}")
print("Metrics visualization saved as 'sheep_tracking_metrics.png'")
